# Calling RES with Python
## Pre-Requisite
   * Python 3
   * Support of PySpark

In [1]:
import os

cwd = os.getcwd()
print(cwd)
# Download Material for Rule Exection
!curl -o {cwd}/miniloan-xom.jar https://raw.githubusercontent.com/ODMDev/decisions-on-spark/pythonintegration/data/miniloan/miniloan-xom.jar
!curl -o {cwd}/miniloan-ruleapp.jar https://raw.githubusercontent.com/ODMDev/decisions-on-spark/pythonintegration/data/miniloan/miniloan-ruleapp.jar
# Download ODM Library
!curl -o {cwd}/j2ee_connector-1_5-fr.jar http://159.122.179.123:31329/download/lib/ODM8920/j2ee_connector-1_5-fr.jar
!curl -o {cwd}/jrules-engine.jar http://159.122.179.123:31329/download/lib/ODM8920/jrules-engine.jar
!curl -o {cwd}/jrules-res-execution.jar http://159.122.179.123:31329/download/lib/ODM8920/jrules-res-execution-memory.jar

os.environ['PYSPARK_SUBMIT_ARGS'] = "--jars local:"+cwd+"/miniloan-ruleapp.jar,local:"+cwd+"/miniloan-xom.jar,local:"+cwd+"/jrules-engine.jar,local:"+cwd+"/j2ee_connector-1_5-fr.jar,local:"+cwd+"/jrules-res-execution.jar pyspark-shell"


import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
sc = SparkSession.builder.getOrCreate()


/home/jovyan/work
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4146  100  4146    0     0  14967      0 --:--:-- --:--:-- --:--:-- 14967
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14663  100 14663    0     0  54712      0 --:--:-- --:--:-- --:--:-- 54712
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 32796  100 32796    0     0   307k      0 --:--:-- --:--:-- --:--:--  310k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.7M  100 13.7M    0     0  1378k      0  0:00:10  0:00:10 --:--:-- 1177k
  % Total    % Received % Xferd  A

In [2]:
def CreateODMSession(sc):
    factoryConfig = sc._jvm.ilog.rules.res.session.IlrJ2SESessionFactory.createDefaultConfig()
    xuConfig = factoryConfig.getXUConfig();
    xuConfig.setLogAutoFlushEnabled(True);
    xuConfig.getPersistenceConfig().setPersistenceType(sc._jvm.ilog.rules.res.session.config.IlrPersistenceType.MEMORY);
    xuConfig.getManagedXOMPersistenceConfig().setPersistenceType(sc._jvm.ilog.rules.res.session.config.IlrPersistenceType.MEMORY);
    factory=sc._jvm.ilog.rules.res.session.IlrJ2SESessionFactory(factoryConfig)
    return factory


factory=CreateODMSession(sc)
# Create a session request object
sessionRequest = factory.createRequest()

sessionRequest.setRulesetPath(sc._jvm.ilog.rules.res.model.IlrPath.parsePath("/miniloanruleapp/1.0/miniloanrules/1.0"));
# Ensure latest version of the ruleset is taken into account
sessionRequest.setForceUptodate(True);
# Set the input parameters for the execution of the rules
inputParameters = sc._jvm.java.util.HashMap();

borrower =  sc._jvm.miniloan.Borrower("Smith John", 200,20000)
loan =  sc._jvm.miniloan.Loan()
loan.setAmount(6000000)
loan.setDuration(1000)
loan.setYearlyInterestRate(0.05)

inputParameters["loan"]=loan
inputParameters["borrower"]=borrower

sessionRequest.setInputParameters(inputParameters)
# Create the stateless rule session.
session = factory.createStatelessSession()
# Execute rules
sessionResponse = session.execute(sessionRequest)
# Display the report
outParameters=sessionResponse.getOutputParameters()
print("Decision Executions return this parameters : ")
for key in outParameters:
    print (key, "corresponds to", outParameters[key])
print("Nb of Rules fire : ",outParameters['ilog.rules.firedRulesCount'])
print("Loan Approved : ",outParameters['loan'].isApproved())
print("Loan Message : ",outParameters['loan'].getMessages())

Decision Executions return this parameters : 
ilog.rules.firedRulesCount corresponds to 1
loan corresponds to miniloan.Loan@df01536
Nb of Rules fire :  1
Loan Approved :  False
Loan Message :  ['The loan cannot exceed 1,000,000']
